# Package Installation and Data Reading

In [1]:
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.stats import variation, iqr

In [2]:
def read_csv_files_to_dataframes(relative_path: str) -> dict:
    """
    Read CSV files from a specified relative path and store them in separate DataFrames.

    Args:
        relative_path (str): The relative path of the directory containing the CSV files.

    Returns:
        dict: A dictionary where keys are the file names (without extension) and values are the corresponding DataFrames.

    Raises:
        FileNotFoundError: If the specified relative path does not exist.

    Example:
        dataframes = read_csv_files_to_dataframes('data_folder')
        df1 = dataframes['file1']  # Access the DataFrame for file1.csv
        df2 = dataframes['file2']  # Access the DataFrame for file2.csv
    """
    data_folder = os.path.join(os.getcwd(), relative_path)

    if not os.path.exists(data_folder):
        raise FileNotFoundError(f"The specified relative path '{relative_path}' does not exist.")

    csv_files = [file for file in os.listdir(data_folder) if file.endswith('.csv')]
    dataframes = {}

    for file in csv_files:
        file_path = os.path.join(data_folder, file)
        df_name = os.path.splitext(file)[0]
        dataframes[df_name] = pd.read_csv(file_path)

 

    return dataframes 

In [3]:
relative_path = "/Users/alexholzer/Desktop/retail_price_prediction/data"
dataframes = read_csv_files_to_dataframes(relative_path)

In [4]:
# create a DataFrame for each file
for file_name in dataframes:
    locals()[file_name] = dataframes[file_name].copy()
    print(file_name)

file_out2
file_out


# Data Familiarization

In [5]:
file_out.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33356 entries, 0 to 33355
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  33356 non-null  int64  
 1   DocumentID  33356 non-null  int64  
 2   Date        33356 non-null  object 
 3   SKU         33356 non-null  int64  
 4   Price       33356 non-null  float64
 5   Discount    33356 non-null  float64
 6   Customer    33356 non-null  int64  
 7   Quantity    33356 non-null  float64
dtypes: float64(3), int64(4), object(1)
memory usage: 2.0+ MB


In [170]:
file_out2.info(memory_usage='True')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29103 entries, 0 to 29102
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  29103 non-null  int64  
 1   InvoiceID   29103 non-null  int64  
 2   Date        29103 non-null  object 
 3   ProductID   29103 non-null  int64  
 4   TotalSales  29103 non-null  float64
 5   Discount    29103 non-null  float64
 6   CustomerID  29103 non-null  int64  
 7   Quantity    29103 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 1.8+ MB


In [6]:
column_mapping = {
    'Unnamed: 0': 'Unnamed: 0',
    'DocumentID': 'InvoiceID',
    'Date': 'Date',
    'SKU': 'ProductID',
    'Price': 'TotalSales',
    'Discount': 'Discount',
    'Customer': 'CustomerID',
    'Quantity': 'Quantity'
}

file_out.columns = [column_mapping[col] for col in file_out.columns]

In [7]:
retail_data_new = dataframes['file_out2']
retail_data_new.drop('Unnamed: 0', axis=1, inplace=True)
retail_data_new['Date']= pd.to_datetime(retail_data_new['Date'])

In [8]:
retail_data_new.head()

,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
0,328,2019-12-27,1684,796.610169,143.389831,185,4
1,329,2019-12-27,524,355.932203,64.067797,185,2
2,330,2019-12-27,192,901.694915,162.305085,230,4
3,330,2019-12-27,218,182.754237,32.895763,230,1
4,330,2019-12-27,247,780.101695,140.418305,230,4


In [189]:
retail_data_new.describe()

,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
count,29103.000000,29103,29103.000000,29103.000000,29103.000000,29103.000000,29103.000000
mean,7221.321445,2020-12-03 12:14:25.395320320,869.953819,2552.679147,451.325294,247.290279,5.440367
min,0.000000,2019-01-02 00:00:00,0.000000,0.000000,-0.004694,0.000000,0.000000
25%,4919.000000,2019-11-13 00:00:00,379.000000,779.661017,137.288136,134.000000,2.000000
50%,7588.000000,2020-09-17 00:00:00,660.000000,1366.101695,244.067797,230.000000,4.000000
75%,9536.000000,2021-12-09 00:00:00,1456.000000,2847.457627,508.658644,349.000000,4.000000
max,14078.000000,2023-03-25 00:00:00,1939.000000,332574.460000,14110.169492,506.000000,250.000000
std,3443.397539,NaN,583.414204,4568.256400,650.359457,138.701207,6.804637


In [19]:
def compute_numeric_measures(df):
    """
    Computes various measures for numeric columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: A DataFrame containing the computed measures for each numeric column.
            The columns of the returned DataFrame include:
            - 'Column Name'
            - 'Count'
            - 'Count null values'
            - 'Sum'
            - 'Mean'
            - 'Mode Frequency'
            - 'Minimum'
            - 'Maximum'
            - 'Variance'
            - 'Standard Deviation'
            - 'Coefficient of Variation (CV)'
            - 'Median Absolute Deviation (MAD)'
            - 'Range'
            - '25% Quantile'
            - '50% Quantile (Median)'
            - '75% Quantile'
            - 'Interquartile Range (IQR)'
            - 'Upper Fence'
            - 'Lower Fence'
            - 'Skewness'
            - 'Kurtosis'
    """
    numeric_columns = df.select_dtypes(include=['int', 'float'])
    measures = []

    for column in numeric_columns:
        column_data = df[column]

        column_measures = {
            'Column Name': column,
            'Count': column_data.count(),
            'Count null values': column_data.isnull().sum(),
            'Sum': column_data.sum(),
            'Mean': column_data.mean(),
            'Mode Frequency': column_data.mode().iloc[0] if len(column_data.mode()) > 0 else None,
            'Minimum': column_data.min(),
            'Maximum': column_data.max(),
            'Variance': column_data.var(),
            'Standard Deviation': column_data.std(),
            'Coefficient of Variation (CV)': variation(column_data),
            'Median Absolute Deviation (MAD)': np.median(np.abs(column_data - column_data.median())),
            'Range': column_data.max() - column_data.min(),
            '25% Quantile': column_data.quantile(0.25),
            '50% Quantile (Median)': column_data.median(),
            '75% Quantile': column_data.quantile(0.75),
            'Interquartile Range (IQR)': iqr(column_data),
            'Upper Fence': column_data.quantile(0.75) + (1.5 * iqr(column_data)),
            'Lower Fence': column_data.quantile(0.25) - (1.5 * iqr(column_data)),
            'Skewness': column_data.skew(),
            'Kurtosis': column_data.kurtosis()
        }

        measures.append(column_measures)

    return pd.DataFrame.from_records(measures)


In [20]:
compute_numeric_measures(retail_data_new)

,Column Name,Count,Count null values,Sum,Mean,Mode Frequency,Minimum,Maximum,Variance,Standard Deviation,...,Median Absolute Deviation (MAD),Range,25% Quantile,50% Quantile (Median),75% Quantile,Interquartile Range (IQR),Upper Fence,Lower Fence,Skewness,Kurtosis
0,InvoiceID,29103,0,2.101621e+08,7221.321445,8716.000000,0.000000,14078.000000,1.185699e+07,3443.397539,...,2211.000000,14078.000000,4919.000000,7588.000000,9536.000000,4617.000000,16461.500000,-2006.500000,-0.203856,-0.640632
1,ProductID,29103,0,2.531827e+07,869.953819,192.000000,0.000000,1939.000000,3.403721e+05,583.414204,...,462.000000,1939.000000,379.000000,660.000000,1456.000000,1077.000000,3071.500000,-1236.500000,0.346144,-1.279308
2,TotalSales,29103,0,7.429062e+07,2552.679147,1016.949153,0.000000,332574.460000,2.086897e+07,4568.256400,...,757.457627,332574.460000,779.661017,1366.101695,2847.457627,2067.796610,5949.152542,-2322.033898,28.999487,1897.567762
3,Discount,29103,0,1.313492e+07,451.325294,0.000000,-0.004694,14110.169492,4.229674e+05,650.359457,...,137.288136,14110.174186,137.288136,244.067797,508.658644,371.370508,1065.714406,-419.767626,5.602114,55.820649
4,CustomerID,29103,0,7.196889e+06,247.290279,230.000000,0.000000,506.000000,1.923802e+04,138.701207,...,106.000000,506.000000,134.000000,230.000000,349.000000,215.000000,671.500000,-188.500000,0.124979,-0.925254
5,Quantity,29103,0,1.583310e+05,5.440367,4.000000,0.000000,250.000000,4.630308e+01,6.804637,...,2.000000,250.000000,2.000000,4.000000,4.000000,2.000000,7.000000,-1.000000,6.541594,98.130143


In [16]:
compute_numeric_measures(retail_data_new)

AttributeError: 'DataFrame' object has no attribute 'concat'